# Going Against the Odds

The notebook is divided into following parts.
1. [Loading Data](#1-loading-the-data)
2. [Describing the Dataset](#2-describing-the-dataset)
3. [Descriptive Statistics for Odds](#3-descriptive-statistics-for-odds)
4. [Margin and Expected Return](#4-margin-and-expected-return)
5. [Monte Carlo Experiments and Strategies](#5-monte-carlo-experiments-and-strategies)

<br><br><br>
## 0. Imports

In [ ]:
import os
import sys

current_directory = os.getcwd()
sys.path.append(current_directory)

from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import seaborn.objects as so
from numba import jit
from tabulate import tabulate

from plot_help_func.plot_heatmap_goals import plot_heatmap_goals
from _constants.data_constants import Bookie, Country, Market
from _utils import filters, plotters, slicers
from logger import LOGGER

In [ ]:
# For jupyter notebooks we set widget and interactivity off. You should run this cell if you want to plot within the notebooks. On default the Qt5Agg Engine is used with interactivity on, which should plot an external figure.
import matplotlib.pyplot as plt

%matplotlib widget
plt.ioff()

<br><br><br>
## 1. Loading the Data

In [ ]:
country = "Germany"
os.environ["path"] = f"{country}.parquet"
path = os.environ.get("path")  # read in the path to the your dataset
df = pd.read_parquet(path, engine="pyarrow")

<br><br><br>
## 2. Describing the Dataset

In [ ]:
# ---- Init filter, plotters and slicers
filter_context = filters.ContextFilter()
slicer_context = slicers.ContextSlicer()

# ---- Descriptive stats of dataframe and info columns
LOGGER.info("Here is some information of the dataframe...")
info = {
    "Countries": list(df.country_oddsportal.dropna().unique()),
    "Dimensions": [df.shape],
    "Match entries from oddsportal": [df.country_oddsportal.notnull().sum()],
    "Match entries from sofascore": [df.country_sofascore.notnull().sum()],
    "Successful merge of oddsportal+sofascore": [
        (df.country_oddsportal.notnull() & df.country_sofascore.notnull()).sum()
    ],
    "Date range of match entries oddsportal": [
        df.agg({"date_oddsportal": ["min", "max"]})["date_oddsportal"]
        .astype(str)
        .tolist()
    ],
    "Date range of match entries sofascore": [
        df.agg({"date_sofascore": ["min", "max"]})["date_sofascore"]
        .astype(str)
        .tolist()
    ],
    "Leagues from oddsportal, N matches": [
        df.league_oddsportal.value_counts().to_string()
    ],
    "Leagues from sofascore, N matches": [
        df.league_sofascore.value_counts().to_string()
    ],
}
df_info = pd.DataFrame.from_dict(info, orient="index", columns=["Information"])
print(tabulate(df_info, headers="keys", tablefmt="fancy_grid"))

In [ ]:
df.head(10)

We now filter the merged dataset:
We will use the *data_oddsportal* and *date_sofascore* column to see if we information from both sites. Odds information is from oddsportal, player related data, statistics etc is from sofascore. General information like results, teams or leagues are from both sites.
- We only look at "Ended" matches which is signaled by the code 100 in the *status_code* column.
- We will filter by the three leagues with the most information
- We will only consider matches after 2008

**NOTE: I use personal filter and slicing classes for the wide dataframe**

In [ ]:
# ---- Filter Dataframe
df_filtered = df  # df_filtered reference to df. CAREFUL! Operations on df_filtered might copy the data and hence eat a lot of RAM.
df_filtered = df_filtered[
    df_filtered[["date_oddsportal", "date_sofascore"]]
    .notnull()
    .all(axis=1)  # Filter only by match entries where the merge was successful
]

In [ ]:
# Filter by status 100 (status 'Ended')
filter_context.filter_strategy = filters.StatusFilter()
df_filtered = filter_context.filtering(df_filtered, status_list=[100])

In [ ]:
# Filter by the top three leagues. We search for the 3 biggest leagues where we have odds from the bookmaker "bet365". The values None, NaN, NaT, and optionally numpy.inf (depending on pandas.options.mode.use_inf_as_na) are considered NA.
leagues = 3
top_leagues = (
    df_filtered.groupby("league_sofascore")["bet365_1x2_1_open"]
    .count()
    .nlargest(leagues)
)

In [ ]:
df_filtered = df_filtered[
    df_filtered.league_sofascore.isin(top_leagues.index.to_list())
]

In [ ]:
# Filter date, get only match entries after 2008
filter_context.filter_strategy = filters.DateFilter()
df_filtered = filter_context.filtering(
    df_filtered, date_start=pd.Timestamp("2008"), date_end=pd.Timestamp("2024")
)

In [ ]:
# We further narrow down the data per season. There is also some broken rows with "liga" within the season.
df_filtered = df_filtered[~df_filtered["season"].isin(["2007/2008", "2022", "liga"])]
df_filtered.reset_index(drop=True, inplace=True)

We now should have most matches during the regular season between 2008 and 2022 for Bundesliga 1-3.

Let's start plotting to get a good overview of basic information.

In [ ]:
# ---- Plotting

# 1. Number of matches per seasons
plotter = (
    plotters.PlotBuilder()
)  # Own plotter class which leverages interactive plotting outside of notebooks with seaborn object API. You can also use seaborn, plotly or raw matplotlib.
plotter.add_data(df_filtered)
plotter.add_xdata(df_filtered.season)
plotter.add_mark(so.Bar(), so.Count(), so.Dodge(), color="league_sofascore")
plotter.add_title(f"Matches per season ({country})")
for container in plotter.axes.containers:
    plotter.axes.bar_label(container, fmt="%.0f")
plotter.set_ylabel("Count")
plotter.set_ylim((0, 500))
sns.move_legend(plotter.axes, "lower center")
plotter.axes.get_legend().set_title("Leagues")

We have around the same number of matches each year. It seems in some years the merge did not work or information is missing.
-  For 1. Bundesliga we see usually 308 matches
-  For 2. Bundesliga we also see 308 matches
-  In the 3. Bundesliga we have more teams resulting in 380 matches playes per season The season 2021/2022 for 3. Bundesliga was not finalized when I parsed the data.

In [ ]:
# ---- plot the distribution of goals as heatmap and display ratio of outcomes
plot_heatmap_goals(df_filtered)
plt.show()

The "home advantage" can be seen clearly with the Home Team winning around 44% of games.

<br><br><br>
## 3. Descriptive Statistics for Odds

**NOTE: We use the *OddsSlicer* Class from *slicers* to get the odds from the wide dataframe**

In [ ]:
slicer_context.slice_strategy = slicers.OddsSlicer()
df_odds = slicer_context.slicing(
    df_filtered,
    bookmaker=["bet365"],  # filter for bookmaker
    odds_market=["1x2"],  # filter for market
    open_closed=["open", "closed"],  # filter for time of odds
    active=True,  # have an column "active" which shows if match was bettable
)
df_odds = df_odds[df_odds["bet365_1x2_active"] == True]

In [ ]:
df_odds.head(10)

In [ ]:
df_odds_closed = df_odds.iloc[:, 1:4].melt(var_name="Outcome", value_name="Odds")
df_odds_closed["Time"] = "Close"
df_odds_open = df_odds.iloc[:, 7:10].melt(var_name="Outcome", value_name="Odds")
df_odds_open["Time"] = "Open"
df_odds_open_close = pd.concat([df_odds_open, df_odds_closed], axis=0).reset_index(
    drop=True
)

In [ ]:
df_odds_open_close["Outcome"] = df_odds_open_close["Outcome"].str.split("_").str[2]

In [ ]:
g = sns.displot(
    df_odds_open_close,
    x="Odds",
    hue="Time",
    kind="kde",
    alpha=0.4,
    col="Outcome",
    fill=True,
)
g.set(xlim=(1, 7), xticks=np.arange(1, 10, 0.5))
for ax in g.axes.flatten():
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.show()

In [ ]:
print(
    tabulate(
        df_odds_open_close.groupby(["Outcome", "Time"])["Odds"].agg(
            ["mean", "median", "std", "skew", pd.DataFrame.kurtosis]
        ),
        headers="keys",
        tablefmt="fancy_grid",
    )
)

<br><br><br>
## 4. Margin and Expected Return

We check the expected return for the bookmaker (hence the margin) to get a better grasp of the competition between bookmakers. We will check if we as a bettor are paying fees on average.

- The margin is percentage by which the implied probability sum exceed 100%.
- The expected return is the percentage of profit that the bookmaker will make on average.

The margin is a function of the expected return and vice versa. We will focus on expected return, because this is ultimately the loss that we can assume as a bettor.


In [ ]:
# ---- slice the odds for all bookies, open and close for the 1x2 market
slicer_context.slice_strategy = slicers.OddsSlicer()
df_odds = slicer_context.slicing(
    df_filtered,
    bookmaker=list(Bookie.BOOKIE),
    odds_market=["1x2"],
    open_closed=["open", "closed"],
    active=True,
)

In [ ]:
# ---- slice the info for our filtered dataset
slicer_context.slice_strategy = slicers.InfoSlicer()
df_info = slicer_context.slicing(df_filtered)

In [ ]:
df_info_and_odds = pd.concat([df_info, df_odds], axis=1)
df_info_and_odds.iloc[:5, :40]

In [ ]:
def calculate_expected_return(df, bookie, status):
    slicer_context.slice_strategy = slicers.OddsSlicer()
    df_odds = slicer_context.slicing(
        df, bookmaker=bookie, odds_market=["1x2"], open_closed=[status], active=True
    )
    df_odds_active = df_odds[df_odds[f"{bookie}_1x2_active"] == True]
    return (1 - (df_odds_active.iloc[:, 1:4] ** -1).sum(axis=1) ** -1) * 100

In [ ]:
# ---- calculate the expected return for each bookie
df_exp_ret = {}
for bookie in Bookie.BOOKIE:
    df_exp_ret[f"{bookie}_1x2_exp_ret_open"] = calculate_expected_return(
        df_info_and_odds, bookie, "open"
    )
    df_exp_ret[f"{bookie}_1x2_exp_ret_closed"] = calculate_expected_return(
        df_info_and_odds, bookie, "closed"
    )
df_exp_ret = pd.concat(df_exp_ret, axis=1)

In [ ]:
df_exp_ret.head(10)

In [ ]:
df_exp_ret.describe()

In [ ]:
df_info_and_exp_ret = pd.concat([df_info, df_exp_ret], axis=1)

We check only a handful of bookmakers which are rather popular. We will filter the dataset at the end. So we can also plot more bookies if necessary.

In [ ]:
bookies_of_interest = [
    "bet365",
    "10Bet",
    "188BET",
    "bet-at-home",
    "Betsson",
    "Betway",
    "Interwetten",
    "bwin",
    "Unibet",
    "NordicBet",
    "Betsafe",
    "Pinnacle",
    "888sport",
    "BetVictor",
    "Betfred",
    "William Hill",
]

In [ ]:
# ---- melt the expected return dataframe
df_exp_ret_long = pd.melt(
    df_info_and_exp_ret,
    id_vars=df_info.columns,
    var_name="bookie",
    value_vars=df_exp_ret.columns,
    value_name="exp_ret_perc",
)

df_exp_ret_long["time"] = df_exp_ret_long["bookie"].str.extract(
    r"_(open|closed)"
)  # get time from bookie string
df_exp_ret_long["bookie"] = df_exp_ret_long["bookie"].str.replace(
    r"_1x2_exp_ret_(open|closed)", "", regex=True
)  # replace now other infos from bookie string
df_exp_ret_long = df_exp_ret_long[
    df_exp_ret_long["bookie"].isin(bookies_of_interest)
]  # only bookies of interest

In [ ]:
# ---- violinplot from with 25%, 50%, 75% quantiles for open & closed odds from bookies of interest
# Note that some bookmakers have less data points regardings odds. Most bookies of interest have odds for >10000 matches.
ax = sns.violinplot(
    data=df_exp_ret_long,
    x="bookie",
    y="exp_ret_perc",
    hue="time",
    kind="violin",
    bw_adjust=0.5,
    split=True,
    cut=0,
    inner="quart",
)
ax.set(ylabel="Expected Return [%]", ylim=[0, 15])
plt.show()

In [ ]:
# ---- group by bookie, year and open/close
df_exp_ret_long["Year"] = df_exp_ret_long["date_sofascore"].dt.year
df_exp_ret_grouped = (
    df_exp_ret_long.groupby(["bookie", "Year", "time"])["exp_ret_perc"]
    .mean()
    .reset_index()
)

In [ ]:
plotter = plotters.PlotBuilder()
plotter.add_xdata(df_exp_ret_grouped["Year"])
plotter.add_ydata(df_exp_ret_grouped["exp_ret_perc"])
plotter.add_data(df_exp_ret_grouped, linestyle="time")
plotter.add_mark(so.Line(), so.Agg("mean"), color="bookie")
plotter.add_mark(so.Line(linewidth=3), so.PolyFit(order=3))
plotter.set_ylabel("Expected Return [%]")
plotter.add_title("Expected Return 1x2 - Mean expected return of bookies over time")
plotter.set_scale(
    x=so.Continuous().tick(every=1),
    y=so.Continuous().tick(every=1),
)
plotter.axes.get_legend().set_title(
    "Mean expected return over Time for", prop={"size": "10"}
)
sns.move_legend(plotter.axes, "upper center")

The 3rd oder fitted line suggest a more competive market over time in the German 1x2 market.

**NOTE: The payouts are dependent on liqudity, thus popularity. The country, league and betting market yield different margins/expected returns. I suggest that further analysis should involve a differentiation on league level and more statitics on deviation.**

<br><br><br>
## 5. Monte Carlo Experiments and Strategies

Monte Carlo Experiments will show us what we can expect of random betting on the 1x2 German market between 2008 to 2022. This will give us insight in the following points:

- Reference level which we compare our strategies against
- Get a feeling how probable it is to have negative return after N bets

What is the default setup?

-  Bookmaker: bet365
-  Leagues: 1. Bundesliga, 2. Bundesliga, 3. Bundesliga
-  Market: 1x2 only
-  Timeframe: season 2007/2008-2021/2022
-  Bankroll: 10000$
-  Stake per Match: 10$
-  N random bets with replace: 1000

In [ ]:
# ---- some configuration for the monte carlo
bookie_to_check = Bookie.BOOKIE[7]  # bet365
initial_balance = 10000
stake_per_game = 10
n_iteration = 100000
n_bets = 1000

In [ ]:
# ---- select the odds for bet365
df_info_and_odds = df_info_and_odds[
    df_info_and_odds[f"{bookie_to_check}_1x2_active"] == True
]  # filter for only active odds for selected bookie
df_mc_set = df_info_and_odds[
    [
        "winner_code_sofascore",
        f"{bookie_to_check}_1x2_1_closed",
        f"{bookie_to_check}_1x2_X_closed",
        f"{bookie_to_check}_1x2_2_closed",
    ]
].reset_index(drop=True)

In [ ]:
df_mc_set.head(10)

In [ ]:
# ---- preallocate the winnings and losses into new columns
outcomes = ["1", "X", "2"]
times = ["closed"]
# negative stake for all columns
for outcome in outcomes:
    for time in times:
        df_mc_set[f"betwin{outcome}_{time}"] = -stake_per_game
# populate columns with winnings
for outcome in outcomes:
    for time in times:
        odds_column = f"{bookie_to_check}_1x2_{outcome}_{time}"
        win_condition = df_mc_set["winner_code_sofascore"] == outcome
        winnings = stake_per_game * (df_mc_set[odds_column] - 1)
        df_mc_set.loc[win_condition, f"betwin{outcome}_{time}"] = winnings

In [ ]:
# ---- Make numpy array from the winnings/losses
arr_mc = df_mc_set[["betwin1_closed", "betwinX_closed", "betwin2_closed"]].to_numpy()
arr_mc.shape

In [ ]:
df_mc_set.head(10)

In [ ]:
# ---- Definition of random betting functions with numba to speed them up
@jit(nopython=True)
def select_random_row(data, num_rows):
    N_rows = data.shape[0]
    selected_rows = np.random.choice(N_rows, num_rows, replace=True)
    return data[selected_rows, :]


@jit(nopython=True)
def select_random_col(data, num_cols):
    N_rows = data.shape[0]
    selected_elements = np.empty(N_rows)
    selected_cols = np.random.choice(num_cols, N_rows, replace=True)
    for row, col in enumerate(selected_cols):
        selected_elements[row] = data[row, col]
    return selected_elements


@jit(nopython=True)
def monte_carlo_sum(
    data,
    iteration: int = 10000,
    batch: int = 1000,
    cols: np.array = np.array([0, 1, 2]),
):  # select from all three cols of the data
    np.random.seed(12345)
    profit_arr = np.empty(iteration)
    for i in range(iteration):
        row_arr = select_random_row(data, batch)
        element_arr = select_random_col(
            row_arr, cols
        )  # not so random, can adjust to 0,1 or 2
        profit = element_arr.sum()
        profit_arr[i] = profit
    return profit_arr

In [ ]:
# ---- n_iterations profit of n_bets
mc_profits = monte_carlo_sum(arr_mc, iteration=n_iteration, batch=n_bets)

In [ ]:
# ---- Make a pandas df again to plot it easier
df_mc = pd.DataFrame(mc_profits, columns=["Profit"])
profit_mean = df_mc["Profit"].mean()

In [ ]:
plotter = plotters.PlotBuilder()
plotter.add_data(df_mc)
plotter.add_xdata(df_mc.Profit)
plotter.add_mark(so.Bars(), so.Hist(stat="probability", binwidth=50))
plotter.set_xlim((-3000, 3000))
plotter.set_xtick(np.arange(-3000, 3000, 500))
plotter.axes.set_xticklabels(plotter.axes.get_xticklabels(), rotation=90)
plotter.set_xlabel("Mean Profit [$]")
plotter.set_ylabel("Probability [%]")
plotter.add_title(f"Monte Carlo - Random Betting on 1x2 Outcomes")
plotter.axes.axvline(profit_mean, color="r", linestyle="--", linewidth=2)
plotter.axes.text(
    profit_mean,
    plotter.axes.get_ylim()[1],
    f" Mean: {profit_mean:.2f}",
    color="r",
    ha="left",
    va="top",
)
plotter.axes.text(
    0.8,
    0.95,
    f"Stake: {stake_per_game}$ per bet\nBets: {n_bets} times\nIterations: {n_iteration}\nUnique Matches: {arr_mc.shape[0]}",
    transform=plotter.axes.transAxes,
    fontsize=10,
    verticalalignment="top",
    bbox=dict(boxstyle="round", facecolor="white", alpha=0.5),
)

We see that with random betting we lose on average 5.9%. So how long will it take that we very safe in the negative with random betting?

In [ ]:
# ---- Get the timeseries array and worst, mid, best quantiles
n_bets_ts = 4000
timeseries_arr = np.empty([n_iteration, n_bets_ts])
for i in range(n_iteration):
    timeseries_balance = initial_balance + np.cumsum(
        select_random_col(select_random_row(arr_mc, n_bets_ts), np.array([0, 1, 2]))
    )
    timeseries_arr[i] = timeseries_balance
quantiles_arr = np.array([1, 50, 99])
quantiles = np.percentile(
    timeseries_arr, quantiles_arr, axis=0
)  # get quantiles of all monte carlo experiments

df_quantiles = pd.DataFrame(quantiles.T, columns=quantiles_arr.astype(str)).melt(
    var_name="Quantile", value_name="Balance"
)

In [ ]:
plotter = plotters.PlotBuilder()
plotter.add_data(df_quantiles)
plotter.add_xdata(pd.Series(np.tile(np.arange(0, n_bets_ts, 1), len(quantiles_arr))))
plotter.add_ydata(df_quantiles["Balance"])
plotter.add_mark(so.Line(), color="Quantile")
plotter.set_xlabel("Number of Bets")
plotter.set_ylabel("Balance")
plotter.add_title(f"Monte Carlo Timeseries - Random Betting on 1x2 Outcomes")
plotter.axes.hlines(
    y=initial_balance, color="r", xmin=-200, xmax=n_bets_ts, linestyle="--", linewidth=2
)

We see that after 3600 bets, so 3.6x our initial balance, we have less than a 1% chance to be positive with random betting.

Let us try an underdog betting strategy. We will only bet on games where the odds for the home team are higher than 1 in comparison to the away team but still lower than 10.

In [ ]:
arr_mc_underdog = df_mc_set.loc[
    (
        df_mc_set[f"{bookie_to_check}_1x2_1_closed"]
        > df_mc_set[f"{bookie_to_check}_1x2_2_closed"] + 1
    )
    & (df_mc_set[f"{bookie_to_check}_1x2_1_closed"] < 10),
    ["betwin1_closed", "betwinX_closed", "betwin2_closed"],
].to_numpy()  # 1 odds higher than away win and no outliners over 10

mc_profits1_underdog = monte_carlo_sum(
    arr_mc_underdog,
    iteration=100000,
    batch=1000,
    cols=np.array([0]),  # only bet on home win
)
df_mc_underdog = pd.DataFrame(mc_profits1_underdog, columns=["Profit"])
profit_mean_underdog = df_mc_underdog["Profit"].mean()

In [ ]:
plotter = plotters.PlotBuilder()
plotter.add_data(df_mc_underdog)
plotter.add_xdata(df_mc_underdog.Profit)
plotter.add_mark(so.Bars(), so.Hist(stat="probability", binwidth=50))
plotter.set_xlim((-3000, 3000))
plotter.set_xtick(np.arange(-3000, 3000, 500))
plotter.axes.set_xticklabels(plotter.axes.get_xticklabels(), rotation=90)
plotter.set_xlabel("Mean Profit [$]")
plotter.set_ylabel("Probability [%]")
plotter.add_title(f"Monte Carlo - Underdog Betting on Home Win")
plotter.axes.axvline(profit_mean_underdog, color="r", linestyle="--", linewidth=2)
plotter.axes.text(
    profit_mean,
    plotter.axes.get_ylim()[1],
    f" Mean: {profit_mean_underdog:.2f}",
    color="r",
    ha="left",
    va="top",
)
plotter.axes.text(
    0.8,
    0.95,
    f"Stake: {stake_per_game}$ per bet\nBets: {n_bets} times\nIterations: {n_iteration}\nUnique Matches: {arr_mc_underdog.shape[0]}",
    transform=plotter.axes.transAxes,
    fontsize=10,
    verticalalignment="top",
    bbox=dict(boxstyle="round", facecolor="white", alpha=0.5),
)

While we observe some profit, caution is advised before adopting an underdog strategy! Strategies evolve, as do the behaviors of both bettors and bookmakers. The higher odds imply a greater risk of bankruptcy, given that only a select few bets may result in a positive outcome. Additionally, this modest profit is specific to the particular bookmaker, leagues, and market examined. It's reasonable to suspect that such a strategy might not maintain its profitability moving forward.